## Genomic Data Science Specialization
Johns Hopkins University
# Course 3 : Algorithms for DNA Sequencing
Module 1 :DNA sequencing, strings and matching
## Practical: Matching artificial reads

In this practical exercise, we will implement the Naive Exact Matching algorithm, as previously presented in lecture, for the purpose of matching artificial reads to the genome.

We shall commence by downloading the genome of the phi X organism.

In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2024-10-11 12:04:39--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 3.163.78.78, 3.163.78.207, 3.163.78.194, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|3.163.78.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa’

phix.fa             100%[===================>]   5.40K  --.-KB/s    in 0s      

2024-10-11 12:04:39 (483 MB/s) - ‘phix.fa’ saved [5528/5528]



In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [3]:
genome = readGenome('phix.fa')

To search, two arguments are needed. The first is P, the pattern, and the second is T, the text to search.

P could be a sequence or read sequence, while T is the genome to match it to.

In [4]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

In [5]:
t = 'AGCTTAGATAGC'
p = 'AG'
naive(p, t)

[0, 5, 9]

A brief pattern will be presented. I'll choose AG and run Naive on the text pattern. P aligns with T three times, yielding zero to two results.

The range is five to seven. The pattern also occurs between positions nine and 11, where T can be any letter in the set AG.

In [15]:
t[0:2]

'AG'

In [16]:
t[5:7]

'AG'

In [17]:
t[9:11]

'AG'

The objective is to generate a set of random reads from the genome sequence.

In [6]:
import random
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

The generation of artificial reads is achieved by extracting sequences from a genome string at random positions.

These reads are then expected to align perfectly with the original genome sequence. To test this approach, a series of reads can be generated using this method.

The genome can be considered the text, whereas the read is the pattern.

In [7]:
# Generate 100 reads of length 100
reads = generateReads(genome, 100, 100)

# Count how many reads match the genome exactly
numMatched = 0
for r in reads:
    matches = naive(r, genome)
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))

100 / 100 reads matched the genome exactly!


## Practical: Matching real reads

In [8]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

--2024-10-11 12:05:16--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 3.163.78.207, 3.163.78.78, 3.163.78.34, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|3.163.78.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: ‘ERR266411_1.first1000.fastq’

ERR266411_1.first10 100%[===================>] 248.42K  1.36MB/s    in 0.2s    

2024-10-11 12:05:16 (1.36 MB/s) - ‘ERR266411_1.first1000.fastq’ saved [254384/254384]



In [9]:
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            fh.readline() # skip base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

The downloaded sequencing reads from the aforementioned file comprise a set of 1,000 reads.

We will now perform the same operation as in Practical Six, using the artificial reads, and align them against the genome to ascertain the number of matches.

In [10]:
import collections
phix_reads = readFastq('ERR266411_1.first1000.fastq')
count = collections.Counter()
for read in phix_reads:
    count.update(read)
count

Counter({'T': 30531, 'A': 28426, 'C': 21890, 'G': 19147, 'N': 6})

In [11]:
numMatched = 0
n = 0
for r in phix_reads:
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

7 / 1000 reads matched the genome exactly!


The reads originate from this genome. Only seven of the 1,000 reads align with the genome. Why would that be the case?

Two potential explanations for this phenomenon have been identified. DNA sequences may not match genomes. Sequencing errors could result in incorrect base interpretation. Sequencing discrepancies may contribute to the lack of alignment. These reads likely aren't aligned due to sequencing errors.

Extract the initial 30 bases to increase the probability of exact alignment. I'll implement a minor modification to extract the first 30 bases and run the program again.

In [12]:
# Now let's try matching just the first 30 bases of each read
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

459 / 1000 reads matched the genome exactly!


459 out of 1,000 matching reads. The figure is under 50%.  This figure is below 1,000. Alignment rate with reference genome is less than 50%.

 Other reasons may explain why not all reads align. The genome is double-stranded, and the reads may originate from either. The problem has been configured to examine only one of the two strands.

The issue is that the reads aren't aligned to both strands of the genome. The discrepancy in matching reads to the genome could be due to the reads aligning to the wrong strand. Aligning the reverse complement would ensure all cases are covered.

In [13]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [14]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

932 / 1000 reads matched the genome exactly!


932 out of 1,000 reads have been identified as matching, but this is not yet 100% accurate.

Once it was determined that a full genome strand had been missed and a prefix read was taken instead of the entire read, the vast majority of reads in the data set were successfully matched.